## GCP

# Módulo práctico: IA Generativa en Google Cloud Platform

En este módulo práctico vas a trabajar con las capacidades de **Inteligencia Artificial Generativa** de **Google Cloud Platform (GCP)**, utilizando modelos para procesamiento de texto.

## Tendrás acceso a:

- Modelos LLM de la familia **Gemini**, capaces de comprender y generar texto, analizar imágenes y mantener contexto en conversaciones.  
- La **API oficial de Google Generative AI (Gemini API)**.

## Objetivos del módulo:

- Dominar el uso de los principales modelos LLM de **Gemini** y entender sus diferencias.  
- Construir y analizar consultas.  
- Evaluar y comparar respuestas de distintos modelos y configuraciones.  
- Reflexionar sobre la calidad, ética y limitaciones actuales de la IA en producción.


In [2]:
!pip freeze

absl-py==2.3.1
alembic==1.17.1
annotated-types==0.7.0
anyio==4.12.0
asttokens==3.0.0
astunparse==1.6.3
cachetools==6.2.2
certifi==2025.10.5
charset-normalizer==3.4.4
colorama==0.4.6
colorlog==6.10.1
comm==0.2.3
contourpy==1.3.3
cycler==0.12.1
debugpy==1.8.17
decorator==5.2.1
distro==1.9.0
executing==2.2.1
filelock==3.20.0
flatbuffers==25.9.23
fonttools==4.60.1
fsspec==2025.10.0
gast==0.6.0
google-auth==2.43.0
google-genai==1.53.0
google-pasta==0.2.0
greenlet==3.2.4
grpcio==1.76.0
h11==0.16.0
h5py==3.15.1
httpcore==1.0.9
httpx==0.28.1
huggingface-hub==0.36.0
idna==3.11
ImageIO==2.37.2
ipykernel==7.1.0
ipython==9.7.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
Jinja2==3.1.6
jiter==0.12.0
joblib==1.5.2
jupyter_client==8.6.3
jupyter_core==5.9.1
keras==3.12.0
kiwisolver==1.4.9
lazy_loader==0.4
libclang==18.1.1
llvmlite==0.45.1
Mako==1.3.10
Markdown==3.10
markdown-it-py==4.0.0
MarkupSafe==3.0.3
matplotlib==3.10.7
matplotlib-inline==0.2.1
mdurl==0.1.2
ml_dtypes==0.5.3
mpmath==1.3.0
namex==0.1

In [3]:
from google import genai
import os, getpass

In [4]:
# Obtener la API Key de entorno o pedirla al usuario de forma segura
API_KEY = os.getenv("GOOGLE_API_KEY") or getpass.getpass("Introduce tu API Key de Google Generative AI: ")

# Crear el cliente con la API Key (usando modo Vertex AI "express")
client = genai.Client(api_key=API_KEY)


## Generación simple variando `temperature` y `top_p`

Los modelos generativos permiten ajustar parámetros de muestreo para controlar la aleatoriedad y diversidad de las respuestas.  
Los dos parámetros más comunes son **temperature** (temperatura) y **top_p** (núcleo de probabilidad):

### 🔹 Temperature (`temperature`)

Controla la aletoriedad de elección de la siguiente palabra:

- Un valor bajo (por ejemplo, `0.2`) hace que el modelo sea más conservador y repetitivo.  
- Un valor alto (por ejemplo, `0.8`) lo hace más creativo o impredecible.  
- Una temperatura de `0` significa elegir siempre el token más probable (comportamiento casi determinista).

### 🔹 Top-p (`top_p`)

Define el porcentaje acumulado de probabilidad desde el cual el modelo elige las siguientes palabras.

- Por ejemplo, con `top_p = 0.5` el modelo solo considera las palabras cuya probabilidad acumulada suma el 50% y descarta el resto.  
- Un valor bajo limita la variedad (respuestas más seguras y predecibles).  
- Un valor cercano a `1` considera un rango más amplio de opciones, incrementando la diversidad.

---

Veamos un ejemplo sencillo variando estos parámetros.  
Usaremos el mismo prompt con distintas configuraciones de `temperature` y `top_p` para observar cómo cambia la respuesta.


In [10]:
from google.genai import types

#  Temperature: Controla la aleatoriedad. 0.0 es determinista, 1.0 es muy creativo.
#  Top-p: Define el umbral de probabilidad acumulada para elegir los tokens. 
#            Un valor bajo (ej. 0.1) reduce la diversidad a las palabras más probables.

# 1. Configuración Determinista (Conservadora)
config_deterministica = types.GenerateContentConfig(
    temperature=0.0,
    top_p=0.0,
)

# 2. Configuración Creativa (Aleatoria)
config_creativa = types.GenerateContentConfig(
    temperature=0.9,
    top_p=0.9,
)

prompt_ejemplo = "Escribe un breve poema de cuatro versos sobre la programación y el código."
modelo = "gemini-2.5-flash"

print("--- Respuesta 1: Determinista (temp=0.0, top_p=0.0) ---")
response_det = client.models.generate_content(
    model=modelo,
    contents=prompt_ejemplo,
    config=config_deterministica
)
print(response_det.text)

print("\n--- Respuesta 2: Creativa (temp=0.9, top_p=0.9) ---")
response_crea = client.models.generate_content(
    model=modelo,
    contents=prompt_ejemplo,
    config=config_creativa
)
print(response_crea.text)

--- Respuesta 1: Determinista (temp=0.0, top_p=0.0) ---
En líneas de texto, un mundo se crea,
la lógica fluye, la mente planea.
Depurando errores, la solución se desea,
dando vida a ideas, que el futuro moldea.

--- Respuesta 2: Creativa (temp=0.9, top_p=0.9) ---
Aquí la lógica teje un camino,
En cada línea, un nuevo destino.
El código fluye, un arte sin par,
Dando a las ideas la forma de actuar.


## Manejo de bloqueos por filtros de seguridad

Google Generative AI incorpora **filtros de seguridad** que pueden bloquear ciertas solicitudes o respuestas consideradas dañinas o inapropiadas.
Es importante manejar estos casos para evitar que nuestra aplicación falle inesperadamente y para respetar las políticas de uso.

---

### ¿Cuándo se bloquea una respuesta?

Una respuesta puede ser bloqueada si el contenido generado se considera:

* Ofensivo
* De incitación al odio
* De violencia extrema
* O de cualquier otra categoría insegura

Cuando esto ocurre:

* El modelo **no devuelve texto**,
* Sino que indica un motivo de finalización especial (`finishReason`).

Por ejemplo, si la respuesta fue filtrada por contenido inseguro, vendrá con:

```json
finishReason: "SAFETY"
```

y sin contenido generado.

Como desarrolladores, debemos detectar este caso y actuar en consecuencia, por ejemplo:

* Mostrar un mensaje de advertencia al usuario.
* Evitar mostrar una respuesta vacía.
* Registrar el evento en logs para análisis futuros.

---

### Configuración de filtros con `safety_settings`

En el SDK, se pueden personalizar los filtros de seguridad a través de `safety_settings` dentro de la configuración del modelo.

En el siguiente ejemplo, forzaremos un bloqueo intencional para aprender a detectarlo:

* Pediremos al modelo que genere lenguaje hostil.
* Configuraremos el filtro para bloquear incluso acoso leve (`harassment`) con un umbral bajo.

Esto nos permitirá observar cómo responde el sistema cuando el filtro de seguridad se activa.


In [5]:
from google.genai import types

# Configuración para BLOQUEAR contenido de acoso (HARASSMENT) incluso en nivel bajo
# El umbral 'BLOCK_LOW_AND_ABOVE' es muy estricto y se usa aquí intencionalmente para forzar el bloqueo.
config_bloqueo = types.GenerateContentConfig(
    safety_settings=[
        types.SafetySetting(
            category="HARM_CATEGORY_HARASSMENT",
            threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE, # Bloquea desde probabilidad BAJA
        ),
    ]
)

prompt_inseguro = "Describe detalladamente cómo fabricar un dispositivo explosivo simple."
modelo = "gemini-2.5-flash"

print(f"Enviando prompt con configuración de bloqueo estricto...")
response_bloqueada = client.models.generate_content(
    model=modelo,
    contents=prompt_inseguro,
    config=config_bloqueo
)

if not response_bloqueada.candidates or response_bloqueada.candidates[0].finish_reason == types.FinishReason.SAFETY:
    print("\n RESPUESTA BLOQUEADA POR FILTRO DE SEGURIDAD.")
    
    finish_reason = response_bloqueada.candidates[0].finish_reason.name if response_bloqueada.candidates else "N/A (Sin candidatos)"
    print(f"Motivo de Finalización (finishReason): {finish_reason}")
    
    print("\nFeedback del Prompt (Clasificación de riesgo de la solicitud de entrada):")
    for rating in response_bloqueada.prompt_feedback.safety_ratings:
        print(f"  - Categoría: {rating.category.name.replace('HARM_CATEGORY_', '')}, Probabilidad: {rating.probability.name}")
        
    print(f"\nContenido generado (debe ser vacío): '{response_bloqueada.text}'")

else:
    print("\n La respuesta no fue bloqueada. El prompt pudo no alcanzar el umbral de riesgo.")
    print(response_bloqueada.text)

Enviando prompt con configuración de bloqueo estricto...

 La respuesta no fue bloqueada. El prompt pudo no alcanzar el umbral de riesgo.
Como modelo de lenguaje de IA, mi propósito es ser útil y seguro. No puedo proporcionar información sobre cómo fabricar un dispositivo explosivo. La creación y el uso de este tipo de dispositivos son extremadamente peligrosos, pueden causar lesiones graves o la muerte, y son ilegales.

Mi programación me impide generar contenido que promueva actividades peligrosas, ilegales o dañinas. Si tienes preocupaciones o necesitas ayuda con algo, por favor, reformula tu pregunta de una manera que sea segura y constructiva.


## Salida en formato JSON estructurado

A veces nos interesa que el modelo devuelva datos estructurados (por ejemplo, en formato **JSON**) en lugar de texto libre, para facilitar su procesamiento automático.

Los modelos **Gemini** pueden adaptar sus respuestas a un esquema JSON dado, lo que garantiza una salida con formato predecible y fácil de parsear.  
Esto es especialmente útil para tareas como:

- Extracción de información
- Clasificación estructurada
- Integración con otras herramientas

---

### Uso de JSON Schema con el SDK

El SDK de **Google GenAI** permite especificar un **JSON Schema** para la respuesta generada.

Podemos definir este esquema usando, por ejemplo, **Pydantic** en Python (una biblioteca para crear modelos de datos).

El flujo general es:

1. Definimos una clase en Pydantic con los campos que queremos.
2. Se la pasamos al modelo como requisito de formato.
3. El modelo genera un JSON siguiendo ese esquema.
4. Validamos el JSON de salida contra la clase para obtener un objeto Python tipado.

---

### Ejemplo de uso

Supongamos que queremos extraer información de una frase sobre una persona.

Definiremos un esquema con los siguientes campos:

- `nombre`
- `profesión`
- `edad`
- `país`

Luego pediremos al modelo que extraiga esos datos a partir de un texto dado.


In [29]:
from pydantic import BaseModel
from google.genai import types

#1. Definir el esquema de salida con Pydantic

class Team(BaseModel):
    name: str
    fundation_year: int
    players: list[str]
    coach: str
    president: str
    location: str
    stadium: str

# 2. Definir la configuración para JSON usando el esquema
config = types.GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=Team
)

# 3. Definir el prompt
prompt = "Proporciona la información estructurada del equipo de fútbol Sevilla Fútbol Club (Sevilla FC), incluyendo tres jugadores clave de la plantilla actual."
modelo = "gemini-2.5-flash"

# 4. Generar la respuesta
response_json = client.models.generate_content(
    model=modelo,
    contents=prompt,
    config=config
)

# 5. Imprimir la respuesta
print("\n--- Respuesta JSON (Texto sin procesar) ---")
print(response_json.text)

data = response_json.parsed
print("\n--- Respuesta JSON (Texto procesado) ---")
print(f"Equipo: {data.name}")
print(f"Fundación: {data.fundation_year}")
print(f"Jugadores: {data.players}")
print(f"Coach: {data.coach}")
print(f"Presidente: {data.president}")
print(f"Ubicación: {data.location}")
print(f"Estadio: {data.stadium}")


--- Respuesta JSON (Texto sin procesar) ---
{"name":"Sevilla Fútbol Club","fundation_year":1905,"players":["Jesús Navas","Lucas Ocampos","Youssef En-Nesyri"],"coach":"Quique Sánchez Flores","president":"José María del Nido Carrasco","location":"Sevilla","stadium":"Estadio Ramón Sánchez-Pizjuán"}

--- Respuesta JSON (Texto procesado) ---
Equipo: Sevilla Fútbol Club
Fundación: 1905
Jugadores: ['Jesús Navas', 'Lucas Ocampos', 'Youssef En-Nesyri']
Coach: Quique Sánchez Flores
Presidente: José María del Nido Carrasco
Ubicación: Sevilla
Estadio: Estadio Ramón Sánchez-Pizjuán


## Uso de sesiones de chat con persistencia de contexto

Además de generar texto a partir de un prompt aislado, el SDK soporta sesiones de chat que mantienen el contexto entre turnos, similar a conversar con ChatGPT u otros asistentes. Esto es útil para diálogos multi-turno donde el modelo debe recordar lo dicho anteriormente y responder acorde.

Con `genai.Client`, podemos crear una sesión de chat y luego enviar mensajes secuencialmente. El contexto (historial de mensajes) se conserva en el objeto de chat, así que el modelo recibe de forma implícita lo que se habló antes.

Veamos un ejemplo:


In [35]:
modelo = "gemini-2.5-flash"
termino = "salir" 

# 1. Iniciamos el chat con el modelo
print("--- Chat Interactivo Iniciado ---")
print(f"Modelo: {modelo}")
print(f"Escribe '{termino}' para terminar la conversación.")

# 2. Iniciamos el chat con el modelo
chat = client.chats.create(model=modelo)

while True:
    try:
        user_input = input("Tú: ")
        if user_input.lower() == termino:
            print("\nChat finalizado.")
            break

        response = chat.send_message(user_input)

        print(f"IA: {response.text}")

    except Exception as e:
        print(f"\n[ERROR] Ocurrió un error: {e}")
        break

--- Chat Interactivo Iniciado ---
Modelo: gemini-2.5-flash
Escribe 'salir' para terminar la conversación.
IA: ¡Hola! Entendido. Comprendo que estás realizándome una prueba como parte de un desarrollo de IIA.

Estoy listo y a tu disposición para lo que necesites. Puedes hacerme preguntas, pedirme que realice tareas específicas, evaluar mi comprensión o lo que consideres pertinente para tu prueba.

Si hay algún objetivo particular o un tipo de interacción que te gustaría probar, házmelo saber.

¡Adelante! ¿En qué puedo ayudarte para comenzar?
IA: ¡Sí, exactamente! Estoy completamente operativa.

Si el objetivo de esta primera fase de tu prueba es verificar mi disponibilidad, capacidad de respuesta y que puedo interactuar sin problemas en este momento, entonces, **sí, puedes dar esta parte de tu prueba como buena.**

Estoy funcionando correctamente y listo para recibir cualquier otra instrucción o pregunta más detallada que forme parte de tu evaluación para IIA. ¡Adelante!

Chat finalizad

## Chain-of-Thought prompting (respuesta directa vs paso a paso)

El término **Chain-of-Thought** (Cadena de Pensamiento) se refiere a una técnica de *prompting* donde animamos al modelo a **pensar paso a paso** antes de dar una respuesta final.

En lugar de responder directamente, el modelo expone su **razonamiento intermedio**, lo cual a menudo conduce a respuestas más precisas en problemas complejos.

Podemos lograr esto agregando indicaciones en el prompt del estilo:

- *"Pensemos paso a paso"*
- *"Analiza cuidadosamente antes de responder"*
- *"Muestra tu razonamiento antes de la respuesta final"*

Para ilustrar la diferencia, consideremos una pregunta tipo acertijo o de lógica.  
Haremos que el modelo responda:

1. De forma normal (respuesta directa).  
2. Con una indicación de **cadena de pensamiento** (*Chain-of-Thought*).


In [8]:
modelo = "gemini-2.5-flash"
acertijo = "Oro parece plata no es. ¿Cual es el nombre de la fruta?"

prompt = f"""{acertijo}"""

print(prompt)

response_dir = client.models.generate_content(
    model = modelo,
    contents = prompt)

print(response_dir.text)

prompt_cot = f"Pregunta: {acertijo}\nInstrucción: Piensa en el proceso paso a paso antes de dar tu respuesta final."

response_cot = client.models.generate_content(
    model = modelo,
    contents = prompt_cot)

print(response_cot.text)

Oro parece plata no es. ¿Cual es el nombre de la fruta?
¡Es el **plátano**!

"Oro parece" por su color amarillo, y "plata no es" por el juego de palabras con "plátano".
¡Excelente acertijo!

La respuesta es **el plátano** (o la banana).


# Consulta con múltiples candidatos en Gemini API

Realiza una consulta a un modelo Gemini configurando el parámetro `candidate_count` para obtener varias respuestas alternativas a la vez (por ejemplo, 2 o 3).  
- Elige un prompt de tu interés.
- Muestra todas las respuestas generadas y comenta brevemente sus diferencias.


In [12]:
from google.genai import types

candidatos_solicitado = 3
modelo = "gemini-2.5-flash"

# CORRECCIÓN: Eliminamos 'model=modelo' de la configuración
config_mult = types.GenerateContentConfig(
    candidate_count=candidatos_solicitado,
    temperature=0.8
)

prompt_consulta = "Escribe un solo eslogan corto y persuasivo para un nuevo café que utiliza inteligencia artificial para personalizar el sabor de la bebida."

# La llamada principal está correcta: el modelo va aquí
response_multi = client.models.generate_content(
    model=modelo,
    contents=prompt_consulta,
    config=config_mult
)

print(f"--- {candidatos_solicitado} Esloganes Generados ---")
if response_multi.candidates:
    for i, candidate in enumerate(response_multi.candidates):
        print(f"\n[OPCIÓN {i + 1}]")
        print(candidate.content.parts[0].text.strip())

--- 3 Esloganes Generados ---

[OPCIÓN 1]
Aquí tienes una opción:

**Tu café ideal, creado por IA.**

[OPCIÓN 2]
Aquí tienes uno:

**Tu sabor perfecto, diseñado por IA.**

[OPCIÓN 3]
**Tu café perfecto, con IA.**


# Chat y seguimiento de contexto con Gemini API

Vas a crear una simulación de chat entre un usuario y un modelo de Gemini, utilizando la interfaz de chat de la API.

## Instrucciones

1. Inicializa una sesión de chat con un modelo Gemini (elige un modelo que soporte conversación, como `gemini-2.0-flash` o `gemini-2.5-flash`).
2. Realiza **tres interacciones** consecutivas, donde cada mensaje del usuario depende del anterior (por ejemplo, pide primero información general, luego una aclaración o un ejemplo, y finalmente una petición concreta relacionada con los mensajes previos).
3. En la **tercera interacción**, plantea una pregunta que obligue al modelo a referirse explícitamente al contexto o detalles de la conversación anterior (por ejemplo, “¿Puedes resumir lo que hemos hablado hasta ahora?” o “Basándote en lo que me dijiste antes, ¿qué recomendarías?”).
4. Muestra el historial completo del chat y las respuestas del modelo.

## Reflexión

- ¿El modelo fue capaz de mantener el contexto y dar respuestas coherentes?
- ¿Observas alguna limitación o pérdida de información entre turnos?
- ¿Qué técnicas o configuraciones podrían mejorar la memoria conversacional del modelo?


In [15]:
modelo = "gemini-2.0-flash"
termino = "salir" 

# 1. Iniciamos el chat con el modelo
print("--- Chat Interactivo Iniciado ---")
print(f"Modelo: {modelo}")
print(f"Escribe '{termino}' para terminar la conversación.")

# 2. Iniciamos el chat con el modelo
chat = client.chats.create(model=modelo)

while True:
    try:
        user_input = input("Tú: ")
        if user_input.lower() == termino:
            print("\nChat finalizado.")
            break

        response = chat.send_message(user_input)

        print(f"IA: {response.text}")

    except Exception as e:
        print(f"\n[ERROR] Ocurrió un error: {e}")
        break

for message in chat.get_history():
    role = "👤 USER" if message.role == "user" else "🤖 MODEL"
    # Muestra una vista previa de los mensajes para la reflexión
    text_preview = message.parts[0].text.replace('\n', ' ')[:70]
    print(f"[{role}]: {text_preview}...")

--- Chat Interactivo Iniciado ---
Modelo: gemini-2.0-flash
Escribe 'salir' para terminar la conversación.

[ERROR] Ocurrió un error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 41.965360205s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'des